# Lets create some fake data first to do some statistics

Here we'll use Python to generate some toy data to demonstrate the two approaches to the problem. Because the measurements are number counts, a Poisson distribution is a good approximation to the measurement process:

In [ ]:
import numpy as np

In [ ]:
from scipy import stats

In [ ]:
from scipy.optimize import minimize

In [ ]:
np.random.seed(200)

In [ ]:
F_true = 100 #true flux, say number of photons detected per sec

In [ ]:
N = 200 # number of measurements

In [ ]:
F = stats.poisson(F_true).rvs(N)  # N measurements of the flux

In [ ]:
#F

In [ ]:
e = np.sqrt(F)  # errors on Poisson counts estimated via square root

Now let's make a simple visualization of the "measured" data:

In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig, ax = plt.subplots()
ax.errorbar(F, np.arange(N), xerr=e, fmt='ok', ecolor='gray', alpha=0.5)
ax.vlines([F_true], 0, N, linewidth=5, alpha=0.5)
ax.set_xlabel("Flux");ax.set_ylabel("measurement number");

In [ ]:
ax.set_xlabel("Flux");ax.set_ylabel("measurement number");

Although we know the true flux in this case "FORGET ABOUT THAT FOR NOW"
The real question is this: given our measurements and errors, what is our best estimate of the true flux?

# Frequentist approach(unbinned liklihood analysis)

Here we will use the classical maximum liklihood approach to find the parameter of interest(Photon flux).Given a single observation $Di=(Fi,ei)$, we can compute the probability of this measurement given the true flux $F_{true}$ and Gaussian errors:

$$P(D_{i}| F_{true}) = \frac{1}{\sqrt{2\pi e_{i}^2}}\exp\Big[\frac{-(F_{i} - F_{true})^2}{2e_{i}^2}\Big] $$

We construct the likelihood function by computing the product of the probabilities for each data point:
$$ L(D | F_{true}) = \prod\limits_{i=1}^n P(D_{i} | F_{true})$$

Here D represents the entire set of measurements.Because the value of the likelihood can become very small, it is often more convenient to instead compute the log-likelihood. Combining the previous two equations and computing the log

$$\log L = -\frac{1}{2} \sum\limits_{i=1}^n \Big[ \log(2\pi e_{i}^2) + \frac {(F_{i} - F_{true})^2}{e_{i}^2}\Big]$$

What we'd like to do is determine $F_{true}$ such that the likelihood is maximized. For this simple problem, the maximization can be done numerically or analytically. I will use python to find the maximum if the function. Let us first see the graph to get an idea what I am doing here:

In [ ]:
F_ParSpace = np.linspace(F_true - 200,F_true + 200,401)

In [ ]:
def loglikihood(Ftrue):
    sum = 0
    for i in range(0, len(F)):
        sum += np.log(2*np.pi*e[i]) + ((F[i] - Ftrue)**2)/(e[i]**2)
    
    return 0.5*sum

In [ ]:
p_x = -1*loglikihood(F_ParSpace)

In [ ]:
plt.plot(F_ParSpace,p_x)

In [ ]:
 opt_fn = lambda S: loglikihood(S)

In [ ]:
result = minimize(opt_fn,1)

In [ ]:
print(result)

In [ ]:
logmax = result.x[0]  # This gives the value of parameter which gives minimum of logliklihood function

In [ ]:
result.fun          # This gives the value of minimum value of function

In [ ]:
import scipy.optimize

In [ ]:
function = lambda x: loglikihood(x) - loglikihood(logmax) - 0.5  # defining function to find the error on the parameter

In [ ]:
S_error = scipy.optimize.brentq(function, logmax, F_true+ 200)  # finding the roots where the liklihood decrease by 0.5

In [ ]:
print("F_estimated = " + str(logmax) + " +/- " + str(S_error - logmax))

Here the 1 Sigma confidence interval says that we are 68 percent confident that the true mean lies in this interval.It does not talk about the probability. if we consider a 95% confidence interval,Then 95% of all the realized 95% confidence intervals calculated in independent measurement problems will contain the actual values of the measurands.

# Bayesian Statistics

Lets see what is a Bayes' Theorem, a fundamental law of probability:

$$P(F_{True} | D) = \frac{P(D | F_{True}) P(F_{True})}{P(D)}$$

If we set the prior P(Ftrue)$\propto$1 (a flat prior), we find

$$P(Ftrue|D) \propto L(D|Ftrue)$$

and the Bayesian probability is maximized at precisely the same value as the frequentist result! 

In [ ]:
def log_prior(Ftrue):
    mmu = 80
    msigma = 1
    if Ftrue < 0:  #out the permissible range Prob is zero and log(Prob) become inf
        return -np.inf
    return -0.5*((Ftrue - mmu)/msigma)**2

In [ ]:
#def log_prior(Ftrue):
 #   return 1  # flat prior

def log_likelihood(Ftrue, F, e):
    return -0.5 * np.sum(np.log(2 * np.pi * e ** 2)
                         + (F - Ftrue) ** 2 / e ** 2)

def log_posterior(Ftrue, F, e):
    return log_prior(Ftrue) + log_likelihood(Ftrue, F, e)

In [ ]:
F_ParSpace = np.linspace(F_true - 50,F_true + 50,401)


In [ ]:
fig,axes = plt.subplots(figsize=(8,6))
prior = [log_prior(S) for S in F_ParSpace]
likelihood = [log_likelihood(S,F,e)for S in F_ParSpace]
posterior = [log_likelihood(S,F,e) + log_prior(S) for S in F_ParSpace]


axes.plot(F_ParSpace, prior, label='Prior')
axes.plot(F_ParSpace, likelihood, label='Likelihood')
axes.plot(F_ParSpace, posterior, label='Posterior')
#plt.yscale('log')
plt.legend()
plt.xlabel('F_true')
plt.ylabel('Log-Likelihood')


In [ ]:
ndim = 1  # number of parameters in the model
nwalkers = 50  # number of MCMC walkers
nburn = 1000  # "burn-in" period to let chains stabilize
nsteps = 2000  # number of MCMC steps to take

# we'll start at random locations between 0 and 2000
starting_guesses = 2000 * np.random.rand(nwalkers, ndim)

import emcee
sampler = emcee.EnsembleSampler(nwalkers, ndim, log_posterior, args=[F, e])
sampler.run_mcmc(starting_guesses, nsteps)

sample = sampler.chain  # shape = (nwalkers, nsteps, ndim)
sample = sampler.chain[:, nburn:, :].ravel()  # discard burn-in points

In [ ]:
plt.hist(sample, bins=50, histtype="stepfilled", alpha=0.3, normed=True)

# plot a best-fit Gaussian
F_fit = np.linspace(F_true-2, F_true+2)
pdf = stats.norm(np.mean(sample), np.std(sample)).pdf(F_fit)

#plt.plot(F_fit, pdf, '-k')
plt.xlabel("F"); plt.ylabel("P(F)")

In [ ]:
print("""
      F_true = {0}
      F_est  = {1:.0f} +/- {2:.4f} (based on {3} measurements)
      """.format(F_true, np.mean(sample), np.std(sample, ddof=1), N))